<center><img src="img/logo_hse_black.jpg"></center>

<h1><center>Natural Language Processing</center></h1>
<h2><center>Week2 - Language Modelling</center></h2>

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,8)

## Полезные ссылки

http://colah.github.io/posts/2015-08-Understanding-LSTMs

http://karpathy.github.io/2015/05/21/rnn-effectiveness/

https://www.youtube.com/watch?v=raGTTBn3eVk

# Vanilla RNN
<center><img src='http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png'></center>

<center><img src='http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-SimpleRNN.png'></center>
<center><img src='http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM2-notation.png'></center>


# Vanilla LSTM

<center><img src='https://chunml.github.io/images/projects/creating-text-generator-using-recurrent-neural-network/LSTM.png'></center>

## Text classification with LSTM

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import re

In [ ]:
!wget https://www.dropbox.com/s/jrhs1gxcql57r73/amazon_cells_labelled.txt?dl=0 -O amazon_cells_labelled.txt

In [ ]:
df_text = pd.read_csv('./amazon_cells_labelled.txt', sep='\t', names=['review', 'label'])

In [ ]:
df_text.head()

Задача классификации отзывов

### Стандартная нормализация текстов без изысков

In [ ]:
import nltk
tokenizer = nltk.tokenize.WordPunctTokenizer()


In [ ]:
# Приводим к нижнему регистру
df_text.loc[:, 'review'] = df_text.review.str.lower()
# Токенизируем
df_text.loc[:, 'review'] = df_text.review.apply(lambda x: tokenizer.tokenize(x))
# Склеиваем через пробел
df_text.loc[:, 'review'] = df_text.review.apply(lambda x: ' '.join(x))

In [ ]:
y = df_text.label.values

In [ ]:
max_fatures = 4000

tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df_text.loc[:, 'review'].values)

X = tokenizer.texts_to_sequences(df_text['review'].values)
X = pad_sequences(X, maxlen=20, padding='post')

In [ ]:
X[:4,:]

In [ ]:
embed_dim = 128
lstm_out = 256

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(lstm_out, dropout=0.2))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', 
              optimizer='adam',
              metrics = ['accuracy'])
print(model.summary())

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size = 0.2, random_state = 123)

batch_size = 32
model.fit(X_train, y_train, epochs = 10, batch_size=batch_size, verbose = 2, validation_data=(X_valid, y_valid))

## Tutorial on text generation
[tensorflow tutorial colab](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/text/text_generation.ipynb)